# StockDataAnalyticsSQL
In this project we are going to execute a few SQL comamnds to check if I'm still capabple of writing advanced SQL analytics queries.

We are using the stocks, outstanding_shares and company_info PostgreSQL tables for that. And the predefiend functions in this project.

Remember these analytical tasks have no higher purpose just training and testing my SQL skills.


In [5]:
import pandas as pd
import datetime

from postgres_connect import pg_conn
from fmp_get_stock_prices import fmp_get_stock_prices, insert_stock_prices
from fmp_get_company_profile import fmp_get_company_profile_data, insert_updated_company_profile

### 1. Rank the companys by market_cap per employee per sector

In [94]:
stocks = ['AAPL', 'MSFT', 'GOOG', 'NFLX', 'TSLA']

query1 = """
SELECT 
    stock.id_stock, 
    symbol,
    sector,
    market_cap, 
    employees, 
    round(market_cap/employees/1000,0) as me,
    RANK() OVER (PARTITION BY sector ORDER BY market_cap/employees DESC) AS rank
FROM fs.stock 
    INNER JOIN fs.company_profile ON stock.id_stock = company_profile.id_stock
WHERE
    symbol IN %s
ORDER BY me desc
;
"""
conn = pg_conn()
cur = conn.cursor()
cur.execute(query1, (tuple(stocks),))
df1 = pd.DataFrame(cur.fetchall(), columns=["id_stock","symbol","sector","market_cap", "employees", "me","rank"])
df1

,id_stock,symbol,sector,market_cap,employees,me,rank
0,11810,NFLX,Communication Services,385028353270,14000,27502,1
1,5328,AAPL,Technology,3186187410000,150000,21241,1
2,5580,MSFT,Technology,2821121070200,228000,12373,2
3,5490,GOOG,Communication Services,2000501177255,183323,10912,2
4,7592,TSLA,Consumer Cyclical,757554790400,125665,6028,1


### 2. Max Stock Price difference last 30-days to 200-day AVG 

In [142]:
query2 = """
WITH avg_200 AS (
    SELECT 
        id_stock,
        AVG(high_price) as avg_price
    FROM 
        fs.stock_price
    WHERE 
        date > CURRENT_DATE - 200 * INTERVAL '1 day'
    GROUP BY 
        id_stock
)
SELECT 
    s.symbol,
    round(AVG(avg_price),2) as acg_price,
    Round(MAX(high_price - avg_price),2) as max_dif,
    Round(MIN(high_price - avg_price),2) as min_dif,
    Round(AVG(high_price - avg_price),2) as avg_dif
FROM 
    fs.stock as s 
    INNER JOIN fs.stock_price as p ON s.id_stock = p.id_stock
    INNER JOIN avg_200 ON avg_200.id_stock = s.id_stock
WHERE
    p.date > CURRENT_DATE - 30 * INTERVAL '1 day'
    AND s.symbol IN %s
GROUP BY
    s.id_stock
ORDER BY
    s.id_stock
;
"""


conn = pg_conn()
cur = conn.cursor()
cur.execute(query2, (tuple(stocks),))
df2 = pd.DataFrame(cur.fetchall(), columns=["symbol","avg_price","max_dif", "min_dif", "avg_dif"])
df2

,symbol,avg_price,max_dif,min_dif,avg_dif
0,AAPL,235.54,14.46,-18.70,4.63
1,GOOG,179.60,8.55,-11.48,-0.84
2,MSFT,424.31,-5.00,-39.09,-22.58
3,TSLA,324.56,42.78,-87.50,-17.91
4,NFLX,855.55,208.95,25.72,133.69


### 3. Price change last 14 days with Partition By

In [157]:
query3 = """

SELECT 
    s.symbol,
    p.date,
    p.close_price,
    change,
    SUM(change) OVER (PARTITION BY s.id_stock ORDER BY date) as sum_change
FROM 
    fs.stock as s 
    INNER JOIN fs.stock_price as p ON s.id_stock = p.id_stock
WHERE
    p.date > CURRENT_DATE - 14 * INTERVAL '1 day'
    AND s.symbol IN %s
ORDER BY
    s.id_stock, date ASC
;
"""
test = """

;
"""


conn = pg_conn()
cur = conn.cursor()
cur.execute(query3, (tuple(stocks[:2]),)) # only first 2 symbols
df3 = pd.DataFrame(cur.fetchall(), columns=["symbol","date","close_price", "change","sum_change"])
df3

,symbol,date,close_price,change,sum_change
0,AAPL,2025-03-03,238.03,-3.76,-3.76
1,AAPL,2025-03-04,235.93,-1.78,-5.54
2,AAPL,2025-03-05,235.74,0.32,-5.22
3,AAPL,2025-03-06,235.33,0.895,-4.325
4,AAPL,2025-03-07,239.07,3.97,-0.355
5,AAPL,2025-03-10,227.48,-8.06,-8.415
6,AAPL,2025-03-11,220.84,-2.97,-11.385
7,AAPL,2025-03-12,216.98,-3.16,-14.545
8,AAPL,2025-03-13,209.68,-6.27,-20.815
9,MSFT,2025-03-03,388.49,-10.33,-10.33
